## Test Calibration

*A notebook wherein some simple test data is subjected to the same
calibration steps as we are applying to real observational data, in order to 
confirm that calibration works exactly as expected.*

The method used for combining images is median, rather than the default, which
is average. This can be readily changed below.

In [1]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct

In [2]:
method = 'median'  # can be 'median' or 'average'

light_exposure = 30 * u.second
dark_exposure = 30 * u.second
flat_exposure = 1 * u.second
bias_exposure = 1 * u.second

In [3]:
light1 = CCDData(np.array([[210, 200],[210, 200]]), unit=u.adu)
light2 = CCDData(np.array([[105, 100],[105, 100]]), unit=u.adu)
lights = [light1, light2]

In [4]:
dark1 = CCDData(np.array([[1, 2],[2, 1]]), unit=u.adu)
dark2 = CCDData(np.array([[2, 4],[4, 2]]), unit=u.adu)
dark3 = CCDData(np.array([[3, 6],[6, 3]]), unit=u.adu)

master_dark = combine([dark1, dark2, dark3], method=method)  # median rigged to be same as average :)

In [5]:
flat1 = CCDData(np.array([[105, 100],[105, 100]]), unit=u.adu)
flat2 = CCDData(np.array([[107, 101],[107, 101]]), unit=u.adu)
flat3 = CCDData(np.array([[109, 102],[109, 102]]), unit=u.adu)

master_flat = combine([flat1, flat2, flat3], method=method)  # median rigged to be same as average :)

In [6]:
bias1 = CCDData(np.array([[1, 0],[1, 0]]), unit=u.adu)
bias2 = CCDData(np.array([[2, 1],[2, 1]]), unit=u.adu)
bias3 = CCDData(np.array([[3, 2],[3, 2]]), unit=u.adu)

master_bias = combine([bias1, bias2, bias3], method=method)  # median rigged to be same as average :)

In [7]:
lights_subtracted = [subtract_dark(light,
                                   master_dark,
                                   data_exposure=light_exposure,
                                   dark_exposure=dark_exposure,
                                   scale=False) for light in lights]

By either method, the first light subtracted is

208 196

206 198

and the second light subtracted is

103 96

101 98

In [8]:
master_flat_subtracted = subtract_dark(master_flat,
                                       master_bias,
                                       data_exposure=flat_exposure,
                                       dark_exposure=bias_exposure,
                                       scale=False)

By either method, the flat subtracted is

105 100

105 100

In [9]:
lights_calibrated = [flat_correct(l, master_flat_subtracted) for l in lights_subtracted]

We expect the calibrated lights to be

208/105  196/100

206/105  198/100

and

103/105  96/100

101/105  98/100

Numerically, that is

1.9810  1.9600

1.9619  1.9800

and

0.9810  0.9600

0.9619  0.9800

But these need to be multiplied by the mean of the flat field, which is 102.5, and we get

203.0476  200.9000

201.0952  202.9500

and

100.5476  98.4000

98.5952  100.4500

In [10]:
lights_calibrated

[CCDData([[203.04761905, 200.9       ],
          [201.0952381 , 202.95      ]], unit='adu'),
 CCDData([[100.54761905,  98.4       ],
          [ 98.5952381 , 100.45      ]], unit='adu')]